In [1]:
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from model.model import BirdClassificationModel

In [3]:
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
test_datasets = datasets.ImageFolder(
    '/home/kusan/bird/seendataset(416)/bird dataset/val',
    transform=transform,
)

In [5]:
batch_size = 64
test_dataloader = DataLoader(test_datasets, shuffle=False, batch_size=batch_size)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
output_dim=25
model_with_state_dict = BirdClassificationModel(output_dim).to(device)
state_dict = torch.load('/home/kusan/bird/Final Model/bird_classification_model(halka-small).pth')
model_with_state_dict.load_state_dict(state_dict)

<All keys matched successfully>

In [9]:
def model_prediction(model, test_dataloader, device='cuda'):
    model.eval()
    targets = None
    predictions = None

    with torch.no_grad():
        for images, labels in tqdm(test_dataloader):
            images = images.to(device)
            labels = labels.to(device)

            results = model(images)
            # If results is a tuple, unpack it here
            if isinstance(results, tuple):
                results = results[0]  # Select the output that contains the predictions

            if targets is None:
                targets = labels
            else:
                targets = torch.concat([targets, labels])
            
            if predictions is None:
                predictions = results
            else:
                predictions = torch.concat([predictions, results])

    return targets, predictions


In [11]:
def evaluation(model, test_dataloader, device='cuda'):
    # Model Complexity Computation
    st = time.time()
    targets, predictions = model_prediction(model, test_dataloader, device)
    et = time.time()
    elapsed_time = et - st
    pytorch_total_params = sum(p.numel() for p in model.parameters())

    # Apply softmax to get probabilities
    predictions_probs = F.softmax(predictions, dim=1)

    # Convert tensors to numpy arrays
    predictions_argmax_array = predictions_probs.argmax(1).cpu().numpy()
    predictions_array = predictions_probs.cpu().numpy()
    targets_array = targets.cpu().numpy()

    # Evaluation Metrics Computation
    accuracyScore = accuracy_score(targets_array, predictions_argmax_array)
    precisionScore = precision_score(targets_array, predictions_argmax_array, average='macro')
    recallScore = recall_score(targets_array, predictions_argmax_array, average='macro')
    f1Score = f1_score(targets_array, predictions_argmax_array, average='macro')
    roc_auc = roc_auc_score(targets_array, predictions_array, average='macro', multi_class='ovr')
    crossentropyloss = CrossEntropyLoss()(predictions, targets).item()

    result_dict = {
        'accuracy_score': accuracyScore,
        'precision_score': precisionScore,
        'recall_score': recallScore,
        'f1_score': f1Score,
        'roc_auc_score': roc_auc,
        'cross_entropy_loss': crossentropyloss,
        'inference_time': elapsed_time,
        'model_parameters_count': pytorch_total_params
    }
    return result_dict

In [12]:
result = evaluation(model_with_state_dict, test_dataloader)

100%|██████████| 235/235 [03:46<00:00,  1.04it/s]


In [13]:
team_name = str(input("Enter Team Name: "))

In [14]:
import json

with open(team_name + ".json", 'w') as file:
    json.dump(result, file)